<a href="https://colab.research.google.com/github/OmarMohammed88/ML_Task/blob/master/Task_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import seaborn as sp 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [0]:
data=pd.read_csv('jobs_data.csv')

Cleaning Data

In [0]:
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

def black_txt(token):
    return  token not in stop_words_ and token not in list(string.punctuation)  and len(token)>2   
  
def clean_txt(text):
  clean_text = []
  clean_text2 = []
  text = re.sub("'", "",text)
  text=re.sub("(\\d|\\W)+"," ",text) 
  text = text.replace("nbsp", "")
  clean_text = [ wn.lemmatize(word, pos="v") for word in word_tokenize(text.lower()) if black_txt(word)]
  clean_text2 = [word for word in clean_text if black_txt(word)]
  return " ".join(clean_text2)

# Clean Text Data

In [0]:
data['industry'] = data['industry'].map(str).apply(clean_txt)
data['title'] = data['title'].map(str).apply(clean_txt)
data['jobFunction'] = data['jobFunction'].map(str).apply(clean_txt)



# Vectorizing Columns

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
# Vect_title = tfidf_vectorizer.fit_transform((data['title'])) 
# Vect_industry=tfidf_vectorizer.fit_transform((data['industry']))
# Vect_job_funct=tfidf_vectorizer.fit_transform((data['jobFunction']))

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer()



Applying Cosine Similarity

In [0]:
Merge_data=pd.DataFrame()


Vectorize All Text

In [0]:
Merge_data['text']=data['industry']+data['jobFunction']+data['title']

Vectorize_Merge = tfidf_vectorizer.fit_transform((Merge_data['text'])) 
# Count_Text = count_vectorizer.fit_transform((Merge_data['text']))

In [0]:


user_q = data['title'].iloc[[1]]
user_tfidf = tfidf_vectorizer.transform(user_q)


In [0]:
from sklearn.metrics.pairwise import cosine_similarity
cos_similarity_countv = map(lambda x: cosine_similarity(user_tfidf, x),Vectorize_Merge)



In [0]:
output2=list(cos_similarity_countv)

Evaluating The Recomented

In [0]:
def get_recommendation(top, df_all, scores):
  recommendation = pd.DataFrame(columns = ['title', 'jobFunction', 'industry','score'])
  count = 0
  for i in top:
      recommendation.at[count, 'title'] = u
      recommendation.at[count, 'jobFunction'] = df_all['jobFunction'][i]
      recommendation.at[count, 'industry'] = df_all['industry'][i]
      recommendation.at[count, 'score'] =  scores[count]
      count += 1
  return recommendation

In [0]:
u="full stack php developer"
top = sorted(range(len(output2)), key=lambda i: output2[i], reverse=True)[:]
list_scores = [output2[i][0][0] for i in top]
get_recommendation(top, data, list_scores)

,title,jobFunction,industry,score
0,full stack php developer,software development engineer telecom technology,computer software,0.81476
1,full stack php developer,software development engineer telecom technology,computer software,0.81476
2,full stack php developer,engineer telecom technology software development,computer software,0.801467
3,full stack php developer,software development engineer telecom technology,information technology service computer software,0.779702
4,full stack php developer,software development engineer telecom technology,information technology service computer software,0.779702
...,...,...,...,...
10866,full stack php developer,medical healthcare hospitality hotels food ser...,hospitality hotels healthcare medical service,0
10867,full stack php developer,sales retail,market advertise furniture,0
10868,full stack php developer,market advertise media journalism publish,market advertise public relations communicatio...,0
10869,full stack php developer,business development sales retail customer ser...,real estate property management retail healthc...,0


# Using KNN APROEACH INSTEAD OF TIFID

In [0]:
from sklearn.neighbors import NearestNeighbors
n_neighbors = 10
KNN = NearestNeighbors(n_neighbors, p=2)
KNN.fit(Vectorize_Merge)
NNs = KNN.kneighbors(user_tfidf, return_distance=True)

In [0]:
NNs[0][0][1:]


array([0.60867087, 0.63013154, 0.66377331, 0.66377331, 0.66377331,
       0.66377331, 0.66377331, 0.66701193, 0.66701193])

In [0]:
top = NNs[1][0][1:]
index_score = NNs[0][0][1:]

get_recommendation(top, data, index_score)

,title,jobFunction,industry,score
0,full stack php developer,software development engineer telecom technology,computer software,0.608671
1,full stack php developer,engineer telecom technology software development,computer software,0.630132
2,full stack php developer,software development engineer telecom technology,information technology service computer software,0.663773
3,full stack php developer,software development engineer telecom technology,information technology service computer software,0.663773
4,full stack php developer,software development engineer telecom technology,information technology service computer software,0.663773
5,full stack php developer,software development engineer telecom technology,information technology service computer software,0.663773
6,full stack php developer,software development engineer telecom technology,information technology service computer software,0.663773
7,full stack php developer,software development engineer telecom technology,computer software information technology service,0.667012
8,full stack php developer,software development engineer telecom technology,computer software information technology service,0.667012
